In [101]:
import pandas as pd 
import numpy as np
import datetime
import pickle
import us
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score

# model parameters
reg = 1000
max_depth = 5

# Load election result data and COVID-19 numbers per congressional district

In [96]:
house_results = pd.read_csv("vote_results/clean_csv_11_23/house.csv")
house_results.head()

,Unnamed: 0,state_id,STATEFP,seat_id,CD114FP,d_votes,r_votes,other_votes,d_vote_share,r_vote_share,other_vote_share,d_won,r_won,other_won
0,0,AK,2,1,1,159765,191568,0,0.454740,0.545260,0.000000,0,1,0
1,1,AL,1,1,1,116949,211825,0,0.355712,0.644288,0.000000,0,1,0
2,2,AL,1,2,2,105286,197996,287,0.346827,0.652227,0.000945,0,1,0
3,3,AL,1,3,3,104595,217384,0,0.324850,0.675150,0.000000,0,1,0
4,4,AL,1,4,4,56237,261553,0,0.176963,0.823037,0.000000,0,1,0


## Load election predictions (based on 2016 election results)

In [97]:
# Get predictions for each congressional district
housedist_to_party = pickle.load(open("baselines.pkl", "rb"))[1]

# Map state name to STATEFP
state_to_statefp = {}

for i, row in house_results.iterrows():
    state = us.states.lookup(row['state_id']).name.lower()
    state_to_statefp[state] = row['STATEFP']

statefp_to_state = {v: k for k, v in state_to_statefp.items()}

## Load COVID-19 congressional district data from Harvard + Microsoft AI for Health

In [98]:
# Get COVID CD data
covid_cd = pd.read_csv("data/clean-covid-cd.csv")
covid_cd["Date"] = pd.to_datetime(covid_cd["Date"])
covid_cd.head()

,Date,STATEFP,state_id,StateName,CD114FP,incremental_cases,incremental_deaths
0,2020-01-22,1,AL,Alabama,1,0.0,0.0
1,2020-01-23,1,AL,Alabama,1,0.0,0.0
2,2020-01-24,1,AL,Alabama,1,0.0,0.0
3,2020-01-25,1,AL,Alabama,1,0.0,0.0
4,2020-01-26,1,AL,Alabama,1,0.0,0.0


## Merge DataFrames to include COVID cases, COVID deaths, and election predictions

In [110]:
# Prepare COVID CD numbers in 5 sets
covid_datasets = list()
# 1. Up to Election Day (Nov. 3rd) by congressional district
covid_election_day = covid_cd.groupby(["STATEFP", "CD114FP", "state_id"]).sum()
covid_election_day = covid_election_day.add_suffix("_total").reset_index()
covid_election_day["CD114FP"][covid_election_day["CD114FP"] == 0] = 1
covid_datasets.append(covid_election_day)

# 2-5. Excluding 1-2-3-4 weeks (respectively) worth of COVID data
for i in range(1, 5):
    excluded_weeks = datetime.timedelta(days=7*i)
    cut = datetime.datetime(2020, 11, 3) - excluded_weeks
    covid_data = covid_cd[covid_cd["Date"] <= cut]
    covid_data = covid_data.groupby(["STATEFP", "CD114FP", "state_id"]).sum()
    covid_data = covid_data.add_suffix("_total").reset_index()
    covid_data["CD114FP"][covid_data["CD114FP"] == 0] = 1
    covid_datasets.append(covid_data)

In [111]:
house_datasets = list()

for dataset in covid_datasets:
    # Merge COVID CD data with house results
    house_results["covid_total"] = None
    house_results["covid_deaths"] = None
    house_results["d_predicted"] = None
    house_results["r_predicted"] = None

    for i, row in dataset.iterrows():
        statefp = row["STATEFP"]
        if statefp == 11: # skip DC
            continue
        cd = row["CD114FP"]
        predicted_winner = housedist_to_party[statefp_to_state[statefp] + str(cd)]
        house_results["d_predicted"][(house_results["STATEFP"] == statefp) & (house_results["CD114FP"] == cd)] = 1 if predicted_winner == "democrat" else 0
        house_results["r_predicted"][(house_results["STATEFP"] == statefp) & (house_results["CD114FP"] == cd)] = 1 if predicted_winner == "republican" else 0
        house_results["covid_total"][(house_results["STATEFP"] == statefp) & (house_results["CD114FP"] == cd)] = row["incremental_cases_total"]
        house_results["covid_deaths"][(house_results["STATEFP"] == statefp) & (house_results["CD114FP"] == cd)] = row["incremental_deaths_total"]
    house_datasets.append(house_results)

## 1. Predictions with COVID-19 data AND past election results

In [118]:
for index, house_dataset in enumerate(house_datasets):
    # Get predictions with covid
    features = pd.DataFrame()
    features["covid_total"] = house_dataset["covid_total"]
    features["covid_deaths"] = house_dataset["covid_deaths"]
    # Getting d_predicted because d_predicted = ~r_predicted
    features["d_predicted"] = house_dataset["d_predicted"]

    # Getting d_won because d_won = ~r_won
    labels = house_dataset["d_won"]

    train_X, test_X, train_y, test_y = train_test_split(features, labels, test_size=0.33)

    log_reg = LogisticRegression(C=reg, class_weight="balanced").fit(train_X, train_y)
    rdm_for = RandomForestClassifier(max_depth=max_depth, class_weight="balanced").fit(train_X, train_y)
    log_reg_score = log_reg.score(test_X, test_y)
    rdm_for_score = rdm_for.score(test_X, test_y)
    log_reg_f1score = f1_score(test_y, log_reg.predict(test_X))
    rdm_for_f1score = f1_score(test_y, rdm_for.predict(test_X))
    log_reg_auc = roc_auc_score(test_y, log_reg.predict_proba(test_X)[:, 1])
    rdm_for_auc = roc_auc_score(test_y, rdm_for.predict_proba(test_X)[:, 1])

    print(f"--------- COVID DATA EXCLUDING {index} WEEK(S) PRIOR TO NOV 3 ---------")
    print("Accuracy LR: {0}, RF: {1}".format(round(log_reg_score, 5), round(rdm_for_score, 5)))
    print("F1 for LR: {0}, RF: {1}".format(round(log_reg_f1score, 4), round(rdm_for_f1score, 4)))
    print("AUC for LR: {0}, RF: {1}\n".format(round(log_reg_auc, 4), round(rdm_for_auc, 4)))

--------- COVID DATA EXCLUDING 0 WEEK(S) PRIOR TO NOV 3 ---------
Accuracy LR: 0.95683, RF: 0.95683
F1 for LR: 0.9625, RF: 0.9625
AUC for LR: 0.9551, RF: 0.9669

--------- COVID DATA EXCLUDING 1 WEEK(S) PRIOR TO NOV 3 ---------
Accuracy LR: 0.94245, RF: 0.93525
F1 for LR: 0.9459, RF: 0.9388
AUC for LR: 0.9595, RF: 0.9636

--------- COVID DATA EXCLUDING 2 WEEK(S) PRIOR TO NOV 3 ---------
Accuracy LR: 0.96403, RF: 0.96403
F1 for LR: 0.9689, RF: 0.9689
AUC for LR: 0.9749, RF: 0.9783

--------- COVID DATA EXCLUDING 3 WEEK(S) PRIOR TO NOV 3 ---------
Accuracy LR: 0.93525, RF: 0.93525
F1 for LR: 0.9302, RF: 0.9302
AUC for LR: 0.9451, RF: 0.9424

--------- COVID DATA EXCLUDING 4 WEEK(S) PRIOR TO NOV 3 ---------
Accuracy LR: 0.64029, RF: 0.95683
F1 for LR: 0.6479, RF: 0.9605
AUC for LR: 0.6773, RF: 0.9603



## 2. Predictions ONLY with past election results

In [119]:
for index, house_dataset in enumerate(house_datasets):
    # Get predictions WITHOUT covid
    features = pd.DataFrame()
    # Getting d_predicted because d_predicted = ~r_predicted
    features["d_predicted"] = house_dataset["d_predicted"]

    # Getting d_won because d_won = ~r_won
    labels = house_dataset["d_won"]

    train_X, test_X, train_y, test_y = train_test_split(features, labels, test_size=0.33)

    log_reg = LogisticRegression(C=reg, class_weight="balanced").fit(train_X, train_y)
    rdm_for = RandomForestClassifier(max_depth=max_depth, class_weight="balanced").fit(train_X, train_y)
    log_reg_score = log_reg.score(test_X, test_y)
    rdm_for_score = rdm_for.score(test_X, test_y)
    log_reg_f1score = f1_score(test_y, log_reg.predict(test_X))
    rdm_for_f1score = f1_score(test_y, rdm_for.predict(test_X))
    log_reg_auc = roc_auc_score(test_y, log_reg.predict_proba(test_X)[:, 1])
    rdm_for_auc = roc_auc_score(test_y, rdm_for.predict_proba(test_X)[:, 1])

    print(f"--------- COVID DATA EXCLUDING {index} WEEK(S) PRIOR TO NOV 3 ---------")
    print("Accuracy LR: {0}, RF: {1}".format(round(log_reg_score, 5), round(rdm_for_score, 5)))
    print("F1 for LR: {0}, RF: {1}".format(round(log_reg_f1score, 4), round(rdm_for_f1score, 4)))
    print("AUC for LR: {0}, RF: {1}\n".format(round(log_reg_auc, 4), round(rdm_for_auc, 4)))

--------- COVID DATA EXCLUDING 0 WEEK(S) PRIOR TO NOV 3 ---------
Accuracy LR: 0.97842, RF: 0.97842
F1 for LR: 0.9804, RF: 0.9804
AUC for LR: 0.9789, RF: 0.9789

--------- COVID DATA EXCLUDING 1 WEEK(S) PRIOR TO NOV 3 ---------
Accuracy LR: 0.96403, RF: 0.96403
F1 for LR: 0.9645, RF: 0.9645
AUC for LR: 0.9642, RF: 0.9642

--------- COVID DATA EXCLUDING 2 WEEK(S) PRIOR TO NOV 3 ---------
Accuracy LR: 0.95683, RF: 0.95683
F1 for LR: 0.9643, RF: 0.9643
AUC for LR: 0.9549, RF: 0.9549

--------- COVID DATA EXCLUDING 3 WEEK(S) PRIOR TO NOV 3 ---------
Accuracy LR: 0.96403, RF: 0.96403
F1 for LR: 0.96, RF: 0.96
AUC for LR: 0.9662, RF: 0.9662

--------- COVID DATA EXCLUDING 4 WEEK(S) PRIOR TO NOV 3 ---------
Accuracy LR: 0.95683, RF: 0.95683
F1 for LR: 0.961, RF: 0.961
AUC for LR: 0.9551, RF: 0.9551



## 3. Predictions with ONLY COVID-19 data

In [120]:
 for index, house_dataset in enumerate(house_datasets):
    # Get predictions with covid

    features = pd.DataFrame()
    features["covid_total"] = house_dataset["covid_total"]
    features["covid_deaths"] = house_dataset["covid_deaths"]

    # Getting d_won because d_won = ~r_won
    labels = house_dataset["d_won"]

    train_X, test_X, train_y, test_y = train_test_split(features, labels, test_size=0.33)

    log_reg = LogisticRegression(C=reg, class_weight="balanced").fit(train_X, train_y)
    rdm_for = RandomForestClassifier(max_depth=max_depth, class_weight="balanced").fit(train_X, train_y)
    log_reg_score = log_reg.score(test_X, test_y)
    rdm_for_score = rdm_for.score(test_X, test_y)
    log_reg_f1score = f1_score(test_y, log_reg.predict(test_X))
    rdm_for_f1score = f1_score(test_y, rdm_for.predict(test_X))
    log_reg_auc = roc_auc_score(test_y, log_reg.predict_proba(test_X)[:, 1])
    rdm_for_auc = roc_auc_score(test_y, rdm_for.predict_proba(test_X)[:, 1])

    print(f"--------- COVID DATA EXCLUDING {index} WEEK(S) PRIOR TO NOV 3 ---------")
    print("Accuracy LR: {0}, RF: {1}".format(round(log_reg_score, 5), round(rdm_for_score, 5)))
    print("F1 for LR: {0}, RF: {1}".format(round(log_reg_f1score, 4), round(rdm_for_f1score, 4)))
    print("AUC for LR: {0}, RF: {1}\n".format(round(log_reg_auc, 4), round(rdm_for_auc, 4)))

--------- COVID DATA EXCLUDING 0 WEEK(S) PRIOR TO NOV 3 ---------
Accuracy LR: 0.68345, RF: 0.69784
F1 for LR: 0.6944, RF: 0.7123
AUC for LR: 0.7689, RF: 0.7802

--------- COVID DATA EXCLUDING 1 WEEK(S) PRIOR TO NOV 3 ---------
Accuracy LR: 0.65468, RF: 0.6259
F1 for LR: 0.6712, RF: 0.6533
AUC for LR: 0.7231, RF: 0.7443

--------- COVID DATA EXCLUDING 2 WEEK(S) PRIOR TO NOV 3 ---------
Accuracy LR: 0.61151, RF: 0.61871
F1 for LR: 0.6143, RF: 0.6241
AUC for LR: 0.7313, RF: 0.7572

--------- COVID DATA EXCLUDING 3 WEEK(S) PRIOR TO NOV 3 ---------
Accuracy LR: 0.63309, RF: 0.64029
F1 for LR: 0.6107, RF: 0.5763
AUC for LR: 0.7304, RF: 0.7577

--------- COVID DATA EXCLUDING 4 WEEK(S) PRIOR TO NOV 3 ---------
Accuracy LR: 0.64029, RF: 0.68345
F1 for LR: 0.6094, RF: 0.6508
AUC for LR: 0.7341, RF: 0.7878

